In [ ]:
from netCDF4 import Dataset
from netCDF4 import num2date
import glob, os

import matplotlib.pyplot as plt

import cartopy.feature as cfeature

import cartopy.crs as ccrs

import numpy as np

from matplotlib import patheffects

from siphon.catalog import TDSCatalog
import cartopy.io.shapereader as shpreader


## THREDDS Catalog Data Access

In [ ]:
from metpy.plots import ctables
mycmap = ctables.registry.get_colortable("NWSReflectivity")

# GOES Section

In [ ]:
# Random Library Imports
import subprocess,os,glob,tempfile,re,webbrowser,io,sys,types,urllib,\
time

# Importing Datetime Libraries
from datetime import datetime, date

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Proj 

# Numerical and Scientific Libraries
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo
from metpy.plots.ctables import registry
from metpy.plots import add_timestamp

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# More Image Manipulation Options
import PIL
from PIL import Image as PILImage

# Ipyhton Options
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display, Image

# Matplotlib Plotting Libraries
#import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import patheffects

import ipywidgets as widgets
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

In [ ]:
# Unidata/MetPy Dictionary of GOES channels

channel_list = {u'1 - Blue Band 0.47 \u03BCm': 1,
                u'2 - Red Band 0.64 \u03BCm': 2,
                u'3 - Veggie Band 0.86 \u03BCm': 3,
                u'4 - Cirrus Band 1.37 \u03BCm': 4,
                u'5 - Snow/Ice Band 1.6 \u03BCm': 5,
                u'6 - Cloud Particle Size Band 2.2 \u03BCm': 6,
                u'7 - Shortwave Window Band 3.9 \u03BCm': 7,
                u'8 - Upper-Level Tropo. WV Band 6.2 \u03BCm': 8,
                u'9 - Mid-Level Tropo. WV Band 6.9 \u03BCm': 9,
                u'10 - Low-Level WV Band 7.3 \u03BCm': 10,
                u'11 - Cloud-Top Phase Band 8.4 \u03BCm': 11,
                u'12 - Ozone Band 9.6 \u03BCm': 12,
                u'13 - Clean IR Longwave Band 10.3 \u03BCm': 13,
                u'14 - IR Longwave Band 11.2 \u03BCm': 14,
                u'15 - Dirty Longwave Band 12.3 \u03BCm': 15,
                u'16 - CO2 Longwave IR 13.3 \u03BCm': 16}

In [ ]:
# Set the date you want to convert
dt = datetime(2019, 8, 17,0,0)
#dt = now

# Start of year for reference
d0 = datetime(2019, 1, 1,0,0)

# Find the difference and add one to get the day number of the calander year
delta = dt - d0
Julian_Day = delta.days+1
if Julian_Day < 100:
    Julian_Day = "0"+str(Julian_Day)
    if int(Julian_Day) < 10:
        Julian_Day = "0"+str(Julian_Day)

Year = str('{0:%Y}'.format(dt))
Month = str('{0:%m}'.format(dt))
Day = str('{0:%d}'.format(dt))
Hour = str('{0:%H}'.format(dt))
Minute = str('{0:%m}'.format(dt))

#'{0:%Y}'.format(dt)+"-"+'{0:%m}'.format(dt)+"-"+'{0:%d}'.format(dt)+"-"+'{0:%H}'.format(dt)
print("date: ",Year+"-"+Month+"-"+Day+"-"+Hour+"-"+Minute)

# Julian day (Day)
print("Julian number: ",Julian_Day)

# Set the location for the data files. 
* If it doesn't exist already, make a new folder

In [ ]:
# Where the data is supposed to be
GOES_sample_path = '/Users/chowdahead/Downloads/GOES_Data/'+str(Year)+'/'+str(Julian_Day)+'/' # Use full path to file
print("Data path:",GOES_sample_path)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(GOES_sample_path):
    os.makedirs(GOES_sample_path)

In [ ]:
!rclone --include "*C13*" ls publicAWS:noaa-goes16/ABI-L1b-RadC/{Year}/{Julian_Day}/21/

In [ ]:
Hour = 0

In [ ]:
for i in range(0,1):
    if i < 10:
        Hour = "0"+str(i)
    else:
        Hour = i
    ! rclone --include "*C13*" copy publicAWS:noaa-goes16/ABI-L1b-RadC/{Year}/{Julian_Day}/{Hour}/ /Users/chowdahead/Downloads/GOES_Data/{Year}/{Julian_Day}/

# Change to the data directory    
    ! cd /Users/chowdahead/Downloads/GOES_Data/{Year}/{Julian_Day}/

In [ ]:
%%time
# rclone AWS data access via remote server


# Copy all the times for this hour for all 16 channels
# Day must be Julian day number
! rclone --include "*C13*" copy publicAWS:noaa-goes16/ABI-L1b-RadC/{Year}/{Julian_Day}/{Hour}/ /Users/chowdahead/Downloads/GOES_Data/{Year}/{Julian_Day}/

# Change to the data directory    
! cd /Users/chowdahead/Downloads/GOES_Data/{Year}/{Julian_Day}/

In [ ]:

    
# Set a path to save the plots with string format for the date to set the month and day
# This will be where the images will be saved
save_path = "/Users/chowdahead/Desktop/Weather_Blog/GOES/"+Year+'/'+str(Julian_Day)+'/'
#save_path = save_path+"IR/"

print("save path:",save_path)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(save_path):
    os.makedirs(save_path)
    

# Old way of getting the files via this website, 
Webpage = "http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi"+\
    "?source=aws&domain=C&product=ABI-L1b-Rad&date="+\
    '{0:%Y}'.format(dt)+"-"+'{0:%m}'.format(dt)+"-"+'{0:%d}'.format(dt)+"&hour="+'{0:%H}'.format(dt)

#webbrowser.open(Webpage)


In [ ]:
GOES_sample_path

In [ ]:
# CONUS

############################################################################################
# Channel 13
GOES16_samples_13 = []
for name in glob.glob(GOES_sample_path+'/OR*RadC*C13*G16*s'+Year+str(Julian_Day)+str(11)+'*.nc'):
    GOES16_samples_13.append(name)
GOES16_samples_13 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13]
GOES16_samples_13 = sorted(GOES16_samples_13, key=lambda x: int(re.sub('\D', '', x)))

############################################################################################
# Channel 14
GOES16_samples_14 = []
for name in glob.glob(GOES_sample_path+'OR*L1b*RadC*C14*G16*s'+Year+str(Julian_Day)+'*.nc'):
    GOES16_samples_14.append(name)
GOES16_samples_14 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_14]
GOES16_samples_14 = sorted(GOES16_samples_14, key=lambda x: int(re.sub('\D', '', x)))


list(GOES16_samples_13)

In [ ]:
# Lon/Lat Box

# [Lon_0, Lon_1, Lat_0, Lat_1]
# CONUS
#extent = [-120., -70, 20., 50.] 

extent = [-120., -70, 20., 50.] 

In [ ]:
cd chowdahead/

In [ ]:
import GOES_IR_ColorMap as IRmap

In [ ]:
IR_cmap = IRmap.IR_Colormap()

In [ ]:
def Map(Year,Month,Day,Hour,Min,path,channel,savepath,extent,my_cmap=None,vmin=None,vmax=None):
    
    from siphon.catalog import TDSCatalog
    top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
    ref = top_cat.catalog_refs['Radar Data']

    new_cat = ref.follow()
    model = new_cat.catalog_refs[4]
    gfs_cat = model.follow()

    ds = gfs_cat.datasets[0]
    subset = ds.subset()
    query_data = subset.query()
        #query_data.lonlat_box(west=-130, east=-50, south=10, north=60)
    from datetime import datetime, timedelta
    #now = datetime.utcnow()
    now = datetime(Year,Month,Day,Hour,Min)
    #print(now)
        # Allow for NetCDF files
    query_data.accept('netcdf4')
    #query_data.lonlat_box(west=-129, east=-64, south=21, north=48)
    query_data.time(now)
    query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

    # Finally attempt to access the data
    data = subset.get_data(query_data)
    #print(list(data.variables))

    time = data.variables['time']
    lon = data.variables['lon']
    lat = data.variables['lat']
    dtime = num2date(time[:],time.units)
    X = data.variables['x'][:]
    Y = data.variables['y'][:]

    Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
    print(Time)
    reflec_data = data.variables['Base_reflectivity_surface_layer']
    reflec = data.variables['Base_reflectivity_surface_layer'][:]

    
    
    fig = plt.figure(figsize=(17., 15.))
    fig.subplots_adjust(hspace=0.06)
    fig.subplots_adjust(wspace=0.02)
    #colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]
    #cbaxes = fig.add_axes(colorbar_axis) 
    reader = shpreader.Reader('/Users/chowdahead/Downloads/countyl010g_shp_nt00964/countyl010g.shp')

    counties = list(reader.geometries())
    COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())
    
    
    # Add state and country boundaries to plot
    states_boundaries = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

    country_borders = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries',scale='50m', facecolor='none')

    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

    #plotcrs = ccrs.LambertConformal(central_longitude=cent_lon)
    #pl = ccrs.PlateCarree(central_longitude=cent_lon)


    ax = plt.subplot(2,2,1,projection=ccrs.LambertConformal())

    ax.add_feature(states_boundaries,edgecolor='b',linewidth=1)
    ax.add_feature(country_borders,edgecolor='b',linewidth=1)
    #ocean = feat.NaturalEarthFeature(category='physical', scale='50m', name='ocean', facecolor=feat.COLORS['water'])
    #coast = feat.NaturalEarthFeature(category='physical', scale='50m', facecolor='none', name='coastline')
    #state_borders = feat.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lakes', scale='50m', facecolor='none') 
    #land = feat.NaturalEarthFeature(category='physical', scale='50m', name='land', facecolor=feat.COLORS['land'])

    #ax.add_feature(COUNTIES, facecolor='none', edgecolor='blue',alpha=0.5)
    # Lat/Lon Extents [lon0,lon1,lat0,lat1]
    #extent = [-130., -70, 20., 60.]
    ax.set_extent(extent,ccrs.PlateCarree()) 

    #ax.imshow(reflec[0,:,:],cmap='nipy_spectral',interpolation='nearest',
    #          extent=(-129.,-64,22,48),transform=ccrs.PlateCarree()) #transform=ccrs.PlateCarree()
    reflec = np.ma.masked_where(reflec < 9,reflec)

    #ax.contourf(lon[:],lat[:],reflec[0,:,:],cmap='nipy_spectral',transform=ccrs.PlateCarree())
    RADAR_levs = np.arange(0,75,1)
    cs = ax.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),
                       vmin=0, vmax=75,zorder=10)
    
    cbar = plt.colorbar(cs,ax=ax ,orientation='vertical',pad=0.02,aspect=50,fraction=.05) #,aspect=20 pad=0.5
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes('right', size='50%', pad=0.05,map_projection=ccrs.PlateCarree())
    #fig.colorbar(cs, cax=cax, orientation='vertical')
    
    text_time = ax.text(.995, 0.01, 
            Time,
            horizontalalignment='right', transform=ax.transAxes,
            color='white', fontsize=12, weight='bold',zorder=15)

    text_time2 = ax.text(0.005, 0.01, 
                "NWS Radar (dbz)",
                horizontalalignment='left', transform=ax.transAxes,
                color='white', fontsize=12, weight='bold',zorder=15)

    outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_time2.set_path_effects(outline_effect)

    # Plot Title
    #plt.title('NWS Radar (dbz)',loc='left',fontsize=16)
    #plt.title(Time,loc='right',fontsize=16)
    #plt.show()
    #plt.savefig("/Users/chowdahead/Desktop/Radar/Radar_plot_Lamb_data_Lamb_COLORMESH.png",
    #           bbox_inches='tight')
 
    

    
    
#########################################################################################################    
# GOES 
#########################################################################################################

    
    vis_name = path[-76:]
    nc = Dataset(path)
    data = nc.variables['Rad'][:]
            #ch2nc.close()
            #ch2nc = None
            
# For the ch 13 IR plot 
    if channel == 13:
        fk1 = nc.variables['planck_fk1'][0]
        fk2 = nc.variables['planck_fk2'][0]
        bc1 = nc.variables['planck_bc1'][0]
        bc2 = nc.variables['planck_bc2'][0]

        data = (fk2 / ( np.log((fk1 / data) + 1 )) - bc1) / bc2
    else:
        data = data
        
    sat_h = nc.variables['goes_imager_projection'].perspective_point_height
    
    X = nc.variables['x'][:] * sat_h
    Y = nc.variables['y'][:] * sat_h

    proj_var = nc.variables['goes_imager_projection']
    sat_height = proj_var.perspective_point_height
    central_lon = proj_var.longitude_of_projection_origin
    semi_major = proj_var.semi_major_axis
    semi_minor = proj_var.semi_minor_axis

# Set Projection of Plot
    globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
    crs = ccrs.Geostationary(central_longitude=-75.2,satellite_height=sat_height, globe=globe) #central_longitude=central_lon,
    #plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
    plotcrs = ccrs.LambertConformal()
    ax2 = plt.subplot(2,2,2, projection=plotcrs)

    
    ax2.add_feature(states_boundaries,edgecolor='b',linewidth=1)
    ax2.add_feature(country_borders,edgecolor='b',linewidth=1)
    #ax2.add_feature(COUNTIES, facecolor='none', edgecolor='blue',alpha=0.5)
    # Lat/Lon Extents [lon0,lon1,lat0,lat1]
    #extent = [-130., -70, 20., 60.]
    ax2.set_extent(extent,ccrs.PlateCarree())

# Find and convert Julian day to date    
    import datetime as DT
    dt = datetime(2019,1,1)
    #print(vis_name[31:34])
    dtdelta = DT.timedelta(days=int(vis_name[31:34])-1)
    Day = dt + dtdelta
    im_file = vis_name[27:31]+"_"+'{0:%m_%d}'.format(Day)+"_"+vis_name[34:36]+vis_name[36:38]
    title_date = vis_name[27:31]+" "+'{0:%B %d}'.format(Day)+" "+vis_name[34:36]+""+vis_name[36:38]+"Z"
    im_file = vis_name[27:31]+"_"+'{0:%m_%d}'.format(Day)+"_"+vis_name[34:36]+vis_name[36:38]

# Set the plot title    
    #plt.title(title,loc='left',fontdict=font)
    #plt.title(title_date,loc='right',fontdict=font)
    
    
    #text_time = ax2.text(.995, 0.01, 
    #        vis_name[34:36]+":"+vis_name[36:38]+"Z",
    #        horizontalalignment='right', transform=ax.transAxes,
    #        color='white', fontsize=12, weight='bold')
    
    text_time = ax2.text(.995, 0.01, 
        vis_name[27:31]+"-"+'{0:%m-%d}'.format(Day)+" "+vis_name[34:36]+":"+vis_name[36:38]+"Z",
        horizontalalignment='right', transform=ax2.transAxes,
        color='white', fontsize=12, weight='bold',zorder=15)

    text_time2 = ax2.text(0.005, 0.01, 
        title,
        horizontalalignment='left', transform=ax2.transAxes,
        color='white', fontsize=12, weight='bold',zorder=15)

    outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_time2.set_path_effects(outline_effect)


# Plot the image
    img = ax2.pcolormesh(X[:],Y[:],data[:],vmin=vmin,vmax=vmax,transform=crs,cmap=my_cmap)
    #img = ax2.imshow(data[:],origin='upper',vmin=vmin,vmax=vmax,extent=(X.min(), X.max(), Y.min(), Y.max()),
        #interpolation='nearest',transform=crs,cmap=my_cmap) #'Greys'
    
    #divider = make_axes_locatable(ax2)
    #cax = divider.append_axes('right', size='50%', pad=0.05,map_projection=ccrs.PlateCarree())
    #fig.colorbar(img, cax=cax, orientation='vertical');
    #cbar = plt.colorbar(cf, cax=cbaxes, orientation='horizontal', extend='max', extendrect=True,pad=0.2)
    cbar = plt.colorbar(img, ax=ax2,orientation='vertical',pad=0.02,aspect=50,fraction=.05) #,aspect=20 pad=0.5
# Display the figure
    #plt.show()
    
# Set the name for saved figure, and save it 
    
    
    #day_path = savepath+'{0:%m_%d}'.format(Day)+"/"
    #if not os.path.isdir(day_path):
    #    os.makedirs(day_path)
    #print(day_path)
    #fig.savefig(outfile,bbox_inches='tight',dpi=120)
    outfile = "RADAR_GOES"+vis_name[23:25]+"_Ch"+str(channel)+"_"+str(im_file)+".png"
    fig.savefig(save_path+outfile,bbox_inches='tight',dpi=120)
    #plt.savefig("/Users/chowdahead/Desktop/Radar/CONUS_Radar_"\
     #                   +Time[0:10].replace("-","_")+"_"+Time[-6:].replace(":","")+".png",bbox_inches="tight")
   
    
    
    
    plt.close(fig)
    print("next")

In [ ]:
save_path

In [ ]:
Index = GOES16_samples_13.index("/Users/chowdahead/Downloads/GOES_Data/2019/228/OR_ABI-L1b-RadC-M6C13_G16_s20192282001563_e20192282004347_c20192282004435.nc")

In [ ]:
%%time
#Map(Year,Month,Day,Hour,Min,path,channel,title,savepath,extent,my_cmap=None,vmin=None,vmax=None)
i = 5
channel=13
#title = "GOES-"+vis_name[23:25]+" Ch "+str(channel)+" - IR"
title = "GOES16 - Ch13"
#extent = [-120., -70, 20., 50.] 
extent = [-105,-80,25,48]
Map(2019,8,16,20,0,GOES16_samples_13[Index],channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
print('done.')

In [ ]:
len(GOES16_samples_13),list(GOES16_samples_13)

In [ ]:
%%time
for i in range(0,12):
    channel=13
    title = "GOES16 - Ch13"
    #extent = [-120., -70, 20., 50.] 
    extent = [-105,-80,25,48]
    Map(2019,8,17,11,i*5,GOES16_samples_13[i],channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
print('done.')

In [ ]:
############################################################################################
# Channel 13
GOES16_samples_13 = []
for name in glob.glob(GOES_sample_path+'/OR*RadC*C13*G16*s'+Year+str(Julian_Day)+str(12)+'*.nc'):
    GOES16_samples_13.append(name)
GOES16_samples_13 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13]
GOES16_samples_13 = sorted(GOES16_samples_13, key=lambda x: int(re.sub('\D', '', x)))


In [ ]:
%%time
for i in range(0,12):
    channel=13
    title = "GOES16 - Ch13"
    #extent = [-120., -70, 20., 50.] 
    extent = [-105,-80,25,48]
    Map(2019,8,17,12,i*5,GOES16_samples_13[i],channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
print('done.')

In [ ]:
############################################################################################
# Channel 13
GOES16_samples_13 = []
for name in glob.glob(GOES_sample_path+'/OR*RadC*C13*G16*s'+Year+str(Julian_Day)+str(13)+'*.nc'):
    GOES16_samples_13.append(name)
GOES16_samples_13 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13]
GOES16_samples_13 = sorted(GOES16_samples_13, key=lambda x: int(re.sub('\D', '', x)))


In [ ]:
%%time
for i in range(0,12):
    channel=13
    title = "GOES16 - Ch13"
    #extent = [-120., -70, 20., 50.] 
    extent = [-105,-80,25,48]
    Map(2019,8,17,13,i*5,GOES16_samples_13[i],channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
print('done.')

In [ ]:
############################################################################################
# Channel 13
GOES16_samples_13 = []
for name in glob.glob(GOES_sample_path+'/OR*RadC*C13*G16*s'+Year+str(Julian_Day)+str(14)+'*.nc'):
    GOES16_samples_13.append(name)
GOES16_samples_13 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13]
GOES16_samples_13 = sorted(GOES16_samples_13, key=lambda x: int(re.sub('\D', '', x)))


In [ ]:
%%time
for i in range(0,12):
    channel=13
    title = "GOES16 - Ch13"
    #extent = [-120., -70, 20., 50.] 
    extent = [-105,-80,25,48]
    Map(2019,8,17,14,i*5,GOES16_samples_13[i],channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
print('done.')